In [2]:
from nba_api.stats.endpoints import BoxScoreSummaryV2, BoxScoreAdvancedV2, LeagueGameLog
import json
import time

# Step 1: Fetch Game Data
def get_game_data(season='2023-24'):
    """
    Fetch all game data for the specified NBA season and return a list of game IDs.
    """
    game_log = LeagueGameLog(season=season, timeout=10)
    game_data = game_log.get_data_frames()[0]
    return game_data[['GAME_ID', 'MATCHUP']]

# Step 2: Get Player Data for Each Team
def get_players(game_id):
    """
    Fetch players for both teams from a given game ID and categorize them as starters or backups.
    """
    box_score = BoxScoreAdvancedV2(game_id=game_id, timeout=10)
    player_stats = box_score.get_data_frames()[0]

    teams = {}
    for i, team_id in enumerate(player_stats['TEAM_ID'].unique()):
        team_players = player_stats[player_stats['TEAM_ID'] == team_id]
        starters = team_players[team_players['START_POSITION'].notna()]
        backups = team_players[team_players['START_POSITION'].isna()]

        teams[f"team_{i}"] = {
            "starters": {
                row['PLAYER_NAME']: {"minutes": row['MIN']} for _, row in starters.iterrows()
            },
            "backups": {
                row['PLAYER_NAME']: {"minutes": row['MIN']} for _, row in backups.iterrows()
            }
        }
    return teams


def get_game_scores_and_winner(game_id):
    """
    Determine the scores and winner of the game using HOME_TEAM_ID and VISITOR_TEAM_ID.
    """
    summary = BoxScoreSummaryV2(game_id=game_id, timeout=10)

    # Access relevant DataFrames
    game_info = summary.get_data_frames()[0]  # Contains HOME_TEAM_ID and VISITOR_TEAM_ID
    team_data = summary.get_data_frames()[5]  # DataFrame with team stats

    try:
        # Identify home and away teams
        home_team_id = game_info.iloc[0]['HOME_TEAM_ID']
        away_team_id = game_info.iloc[0]['VISITOR_TEAM_ID']

        # Extract scores for home and away teams
        team_1_score = team_data[team_data['TEAM_ID'] == home_team_id]['PTS'].values[0]
        team_0_score = team_data[team_data['TEAM_ID'] == away_team_id]['PTS'].values[0]

        print(game_id)
        print(f"Away Team 0 (ID: {away_team_id}) Score: {team_0_score}, Home Team 1 (ID: {home_team_id}) Score: {team_1_score}")

        # Determine winner
        winner = 0 if team_0_score > team_1_score else 1
        
        return team_0_score, team_1_score, winner
    except Exception as e:
        print(f"Error processing game {game_id}: {e}")
        raise



# Step 4: Process Games and Output JSON
def create_game_json(game_data):
    """
    Create a JSON structure with game data for each game ID, including the game winner.
    """
    all_games = {}
    game_count = 0  # Track the number of processed games

    for _, row in game_data.iterrows():
        game_id = row['GAME_ID']
        try:
            teams = get_players(game_id)
            team_0_score, team_1_score, winner = get_game_scores_and_winner(game_id)
            teams['team_0_score'] = team_0_score
            teams['team_1_score'] = team_1_score
            teams['winner'] = winner  # Add the winner to the game data
            all_games[game_id] = teams
            print(f"Processed game {game_id} - Winner: Team {winner}")
        except Exception as e:
            print(f"Error processing game {game_id}: {e}")

        # Increment game count and pause every 5 games
        game_count += 1
        if game_count % 5 == 0:
            print("Processed 5 games. Taking a 1-minute break...")
            time.sleep(60)  # Wait for 1 minute

    return all_games

# Main Execution
season = '2022-23'
game_data = get_game_data(season=season)
game_json = create_game_json(game_data)

# Save the JSON to a file
with open('game_player_dataa.json', 'w') as f:
    json.dump(game_json, f, indent=4)
print("Game data saved to 'game_player_dataa.json'")

0022200002
Away Team 0 (ID: 1610612747) Score: 109, Home Team 1 (ID: 1610612744) Score: 123
Processed game 0022200002 - Winner: Team 1
0022200002
Away Team 0 (ID: 1610612747) Score: 109, Home Team 1 (ID: 1610612744) Score: 123
Processed game 0022200002 - Winner: Team 1
0022200001
Away Team 0 (ID: 1610612755) Score: 117, Home Team 1 (ID: 1610612738) Score: 126
Processed game 0022200001 - Winner: Team 1
0022200001
Away Team 0 (ID: 1610612755) Score: 117, Home Team 1 (ID: 1610612738) Score: 126
Processed game 0022200001 - Winner: Team 1
0022200005
Away Team 0 (ID: 1610612745) Score: 107, Home Team 1 (ID: 1610612737) Score: 117
Processed game 0022200005 - Winner: Team 1
Processed 5 games. Taking a 1-minute break...
0022200012
Away Team 0 (ID: 1610612743) Score: 102, Home Team 1 (ID: 1610612762) Score: 123
Processed game 0022200012 - Winner: Team 1
0022200008
Away Team 0 (ID: 1610612739) Score: 105, Home Team 1 (ID: 1610612761) Score: 108
Processed game 0022200008 - Winner: Team 1
002220000

0022200036
Away Team 0 (ID: 1610612746) Score: 111, Home Team 1 (ID: 1610612758) Score: 109
Processed game 0022200036 - Winner: Team 0
0022200031
Away Team 0 (ID: 1610612761) Score: 109, Home Team 1 (ID: 1610612748) Score: 112
Processed game 0022200031 - Winner: Team 1
Processed 5 games. Taking a 1-minute break...
0022200034
Away Team 0 (ID: 1610612763) Score: 96, Home Team 1 (ID: 1610612742) Score: 137
Processed game 0022200034 - Winner: Team 1
0022200036
Away Team 0 (ID: 1610612746) Score: 111, Home Team 1 (ID: 1610612758) Score: 109
Processed game 0022200036 - Winner: Team 0
0022200030
Away Team 0 (ID: 1610612738) Score: 126, Home Team 1 (ID: 1610612753) Score: 120
Processed game 0022200030 - Winner: Team 0
0022200032
Away Team 0 (ID: 1610612739) Score: 128, Home Team 1 (ID: 1610612741) Score: 96
Processed game 0022200032 - Winner: Team 0
0022200035
Away Team 0 (ID: 1610612760) Score: 117, Home Team 1 (ID: 1610612743) Score: 122
Processed game 0022200035 - Winner: Team 1
Processed 5

0022200063
Away Team 0 (ID: 1610612745) Score: 101, Home Team 1 (ID: 1610612762) Score: 109
Processed game 0022200063 - Winner: Team 1
0022200060
Away Team 0 (ID: 1610612751) Score: 99, Home Team 1 (ID: 1610612749) Score: 110
Processed game 0022200060 - Winner: Team 1
0022200056
Away Team 0 (ID: 1610612753) Score: 92, Home Team 1 (ID: 1610612739) Score: 103
Processed game 0022200056 - Winner: Team 1
0022200062
Away Team 0 (ID: 1610612759) Score: 122, Home Team 1 (ID: 1610612750) Score: 134
Processed game 0022200062 - Winner: Team 1
0022200062
Away Team 0 (ID: 1610612759) Score: 122, Home Team 1 (ID: 1610612750) Score: 134
Processed game 0022200062 - Winner: Team 1
Processed 5 games. Taking a 1-minute break...
0022200058
Away Team 0 (ID: 1610612766) Score: 131, Home Team 1 (ID: 1610612752) Score: 134
Processed game 0022200058 - Winner: Team 1
0022200057
Away Team 0 (ID: 1610612737) Score: 118, Home Team 1 (ID: 1610612765) Score: 113
Processed game 0022200057 - Winner: Team 0
0022200061


0022200093
Away Team 0 (ID: 1610612753) Score: 105, Home Team 1 (ID: 1610612742) Score: 114
Processed game 0022200093 - Winner: Team 1
Processed 5 games. Taking a 1-minute break...
0022200095
Away Team 0 (ID: 1610612743) Score: 110, Home Team 1 (ID: 1610612747) Score: 121
Processed game 0022200095 - Winner: Team 1
0022200089
Away Team 0 (ID: 1610612764) Score: 94, Home Team 1 (ID: 1610612738) Score: 112
Processed game 0022200089 - Winner: Team 1
0022200091
Away Team 0 (ID: 1610612744) Score: 114, Home Team 1 (ID: 1610612765) Score: 128
Processed game 0022200091 - Winner: Team 1
0022200092
Away Team 0 (ID: 1610612750) Score: 98, Home Team 1 (ID: 1610612759) Score: 107
Processed game 0022200092 - Winner: Team 1
0022200094
Away Team 0 (ID: 1610612745) Score: 109, Home Team 1 (ID: 1610612756) Score: 124
Processed game 0022200094 - Winner: Team 1
Processed 5 games. Taking a 1-minute break...
0022200091
Away Team 0 (ID: 1610612744) Score: 114, Home Team 1 (ID: 1610612765) Score: 128
Processe

0022200108
Away Team 0 (ID: 1610612764) Score: 121, Home Team 1 (ID: 1610612755) Score: 111
Processed game 0022200108 - Winner: Team 0
0022200107
Away Team 0 (ID: 1610612738) Score: 113, Home Team 1 (ID: 1610612739) Score: 114
Processed game 0022200107 - Winner: Team 1
0022200118
Away Team 0 (ID: 1610612744) Score: 129, Home Team 1 (ID: 1610612753) Score: 130
Processed game 0022200118 - Winner: Team 1
Processed 5 games. Taking a 1-minute break...
0022200119
Away Team 0 (ID: 1610612743) Score: 122, Home Team 1 (ID: 1610612760) Score: 110
Processed game 0022200119 - Winner: Team 0
0022200119
Away Team 0 (ID: 1610612743) Score: 122, Home Team 1 (ID: 1610612760) Score: 110
Processed game 0022200119 - Winner: Team 0
0022200118
Away Team 0 (ID: 1610612744) Score: 129, Home Team 1 (ID: 1610612753) Score: 130
Processed game 0022200118 - Winner: Team 1
0022200126
Away Team 0 (ID: 1610612746) Score: 113, Home Team 1 (ID: 1610612759) Score: 106
Processed game 0022200126 - Winner: Team 0
002220012

...

0022201122
Away Team 0 (ID: 1610612749) Score: 126, Home Team 1 (ID: 1610612765) Score: 117
Processed game 0022201122 - Winner: Team 0
0022201122
Away Team 0 (ID: 1610612749) Score: 126, Home Team 1 (ID: 1610612765) Score: 117
Processed game 0022201122 - Winner: Team 0
0022201129
Away Team 0 (ID: 1610612741) Score: 112, Home Team 1 (ID: 1610612746) Score: 124
Processed game 0022201129 - Winner: Team 1
0022201128
Away Team 0 (ID: 1610612750) Score: 119, Home Team 1 (ID: 1610612758) Score: 115
Processed game 0022201128 - Winner: Team 0
0022201127
Away Team 0 (ID: 1610612740) Score: 124, Home Team 1 (ID: 1610612757) Score: 90
Processed game 0022201127 - Winner: Team 0
Processed 5 games. Taking a 1-minute break...
0022201129
Away Team 0 (ID: 1610612741) Score: 112, Home Team 1 (ID: 1610612746) Score: 124
Processed game 0022201129 - Winner: Team 1
0022201126
Away Team 0 (ID: 1610612755) Score: 111, Home Team 1 (ID: 1610612743) Score: 116
Processed game 0022201126 - Winner: Team 1
0022201124

0022201153
Away Team 0 (ID: 1610612737) Score: 107, Home Team 1 (ID: 1610612751) Score: 124
Processed game 0022201153 - Winner: Team 1
0022201155
Away Team 0 (ID: 1610612765) Score: 115, Home Team 1 (ID: 1610612745) Score: 121
Processed game 0022201155 - Winner: Team 1
Processed 5 games. Taking a 1-minute break...
0022201154
Away Team 0 (ID: 1610612752) Score: 130, Home Team 1 (ID: 1610612739) Score: 116
Processed game 0022201154 - Winner: Team 0
0022201153
Away Team 0 (ID: 1610612737) Score: 107, Home Team 1 (ID: 1610612751) Score: 124
Processed game 0022201153 - Winner: Team 1
0022201148
Away Team 0 (ID: 1610612741) Score: 121, Home Team 1 (ID: 1610612766) Score: 91
Processed game 0022201148 - Winner: Team 0
0022201151
Away Team 0 (ID: 1610612753) Score: 116, Home Team 1 (ID: 1610612764) Score: 109
Processed game 0022201151 - Winner: Team 0
0022201157
Away Team 0 (ID: 1610612747) Score: 123, Home Team 1 (ID: 1610612750) Score: 111
Processed game 0022201157 - Winner: Team 0
Processed 

0022201179
Away Team 0 (ID: 1610612749) Score: 140, Home Team 1 (ID: 1610612764) Score: 128
Processed game 0022201179 - Winner: Team 0
0022201183
Away Team 0 (ID: 1610612743) Score: 103, Home Team 1 (ID: 1610612745) Score: 124
Processed game 0022201183 - Winner: Team 1
0022201176
Away Team 0 (ID: 1610612761) Score: 120, Home Team 1 (ID: 1610612766) Score: 100
Processed game 0022201176 - Winner: Team 0
0022201182
Away Team 0 (ID: 1610612737) Score: 123, Home Team 1 (ID: 1610612741) Score: 105
Processed game 0022201182 - Winner: Team 0
0022201184
Away Team 0 (ID: 1610612757) Score: 109, Home Team 1 (ID: 1610612763) Score: 119
Processed game 0022201184 - Winner: Team 1
Processed 5 games. Taking a 1-minute break...
0022201177
Away Team 0 (ID: 1610612748) Score: 118, Home Team 1 (ID: 1610612765) Score: 105
Processed game 0022201177 - Winner: Team 0
0022201185
Away Team 0 (ID: 1610612758) Score: 121, Home Team 1 (ID: 1610612740) Score: 103
Processed game 0022201185 - Winner: Team 0
002220118

0022201203
Away Team 0 (ID: 1610612765) Score: 122, Home Team 1 (ID: 1610612754) Score: 115
Processed game 0022201203 - Winner: Team 0
0022201214
Away Team 0 (ID: 1610612750) Score: 151, Home Team 1 (ID: 1610612759) Score: 131
Processed game 0022201214 - Winner: Team 0
Processed 5 games. Taking a 1-minute break...
0022201213
Away Team 0 (ID: 1610612743) Score: 114, Home Team 1 (ID: 1610612762) Score: 118
Processed game 0022201213 - Winner: Team 1
0022201215
Away Team 0 (ID: 1610612757) Score: 125, Home Team 1 (ID: 1610612746) Score: 136
Processed game 0022201215 - Winner: Team 1
0022201215
Away Team 0 (ID: 1610612757) Score: 125, Home Team 1 (ID: 1610612746) Score: 136
Processed game 0022201215 - Winner: Team 1
0022201213
Away Team 0 (ID: 1610612743) Score: 114, Home Team 1 (ID: 1610612762) Score: 118
Processed game 0022201213 - Winner: Team 1
0022201214
Away Team 0 (ID: 1610612750) Score: 151, Home Team 1 (ID: 1610612759) Score: 131
Processed game 0022201214 - Winner: Team 0
Processed

TypeError: Object of type int64 is not JSON serializable

In [9]:
import pandas as pd
import json
import math
# Step 1: Load Game Data and Player Attributes
def load_game_data(filepath='game_player_data.json'):
    """
    Load game data from a JSON file.
    """
    with open(filepath) as f:
        return json.load(f)

def load_player_attributes(filepath='league.json'):
    """
    Load player attributes from a JSON file.
    """
    with open(filepath) as f:
        player_data = json.load(f)
    return {player['name']: player for player in player_data}

# Step 2: Compute Weighted Average Attributes
def compute_team_averages(players, player_attributes):
    """
    Compute weighted average attributes for a team.
    """
    weighted_sums = {}
    total_weights = 0

    for player_name, player_data in players.items():
        try:
            # Extract minutes played
            minutes_raw = str(player_data['minutes']).split(':')[0]
            minutes_played = int(float(minutes_raw)) if pd.notna(minutes_raw) else 0
        except (ValueError, TypeError):
            print(f"Error parsing MIN for player {player_name}: {player_data['minutes']}")
            minutes_played = 0

        if player_name in player_attributes and minutes_played > 0:
            attributes = player_attributes[player_name]
            stamina = attributes.get('stamina', 1) * 0.01  # Convert stamina to a decimal
            
            # Apply fatigue-based scaling
            scaling_factor = minutes_played / 48.0  # Normalize minutes played to a 48-minute game
            


            weight = scaling_factor * sum(stamina ** i for i in range(1, minutes_played + 1))

            total_weights += weight

            for attr, value in attributes.items():
                if attr in ['name', 'team']:  # Exclude non-numeric attributes
                    continue
                try:
                    value = float(value)
                    if attr not in weighted_sums:
                        weighted_sums[attr] = 0
                    weighted_sums[attr] += value * weight
                except ValueError:
                    print(f"Error parsing attribute {attr} for player {player_name}: {value}")
                    continue

    if total_weights == 0:
        print("No valid players with weights. Returning empty series.")
        return pd.Series(dtype=float)

    # Compute weighted averages
    weighted_averages = {attr: weighted_sum / total_weights for attr, weighted_sum in weighted_sums.items()}
    return pd.Series(weighted_averages)

# Step 3: Create DataFrame
def create_game_dataframe(game_data, player_attributes):
    """
    Create a dataframe with average attributes for both teams, home indicators, and the game outcome.
    """
    rows = []

    for game_id, game_info in game_data.items():
        try:
            team_0_avg = compute_team_averages(game_info['team_0']['starters'], player_attributes)
            team_1_avg = compute_team_averages(game_info['team_1']['starters'], player_attributes)
            winner = game_info['winner']

        

            # Combine team attributes, home indicators, and winner into a single row
            combined = pd.concat([team_0_avg.add_prefix('team_0_'), team_1_avg.add_prefix('team_1_')])
            combined['team_0_is_away'] = 0
            combined['team_1_is_home'] = 1
            combined['winner'] = winner
            rows.append(combined)
            print(f"Processed game {game_id}")
        except Exception as e:
            print(f"Error processing game {game_id}: {e}")

    return pd.DataFrame(rows)

# Main Execution
game_data = load_game_data('combined_game_file2224.json')
player_attributes = load_player_attributes('league.json')
game_df = create_game_dataframe(game_data, player_attributes)

# Save the dataframe to a CSV file
game_df.to_csv('game_attributes_coef.csv', index=False)
print("Dataframe created and saved to 'game_attributes_coef.csv'")

Processed game 0022300061
Processed game 0022300062
Processed game 0022300063
Processed game 0022300068
Processed game 0022300074
Processed game 0022300066
Processed game 0022300072
Processed game 0022300071
Processed game 0022300064
Processed game 0022300069
Processed game 0022300065
Processed game 0022300073
Processed game 0022300067
Processed game 0022300070
Processed game 0022300075
Processed game 0022300076
Processed game 0022300086
Processed game 0022300077
Processed game 0022300081
Processed game 0022300080
Processed game 0022300087
Processed game 0022300078
Processed game 0022300082
Processed game 0022300085
Processed game 0022300084
Processed game 0022300083
Processed game 0022300079
Processed game 0022300089
Processed game 0022300093
Processed game 0022300094
Processed game 0022300091
Processed game 0022300092
Processed game 0022300090
Processed game 0022300088
Processed game 0022300097
Processed game 0022300098
Processed game 0022300095
Processed game 0022300099
Processed ga

Processed game 0022300875
Processed game 0022300877
Processed game 0022300871
Processed game 0022300873
Processed game 0022300876
Processed game 0022300881
Processed game 0022300880
Processed game 0022300884
Processed game 0022300879
Processed game 0022300883
Processed game 0022300882
Processed game 0022300889
Processed game 0022300890
Processed game 0022300888
Processed game 0022300887
Processed game 0022300893
Processed game 0022300885
Processed game 0022300892
Processed game 0022300891
Processed game 0022300886
Processed game 0022300898
Processed game 0022300896
Processed game 0022300901
Processed game 0022300899
Processed game 0022300897
Processed game 0022300900
Processed game 0022300895
Processed game 0022300894
Processed game 0022300908
Processed game 0022300907
Processed game 0022300906
Processed game 0022300904
Processed game 0022300902
Processed game 0022300903
Processed game 0022300905
Processed game 0022300912
Processed game 0022300910
Processed game 0022300913
Processed ga

Dataframe created and saved to 'game_attributes_coef.csv'
